In [1]:
!pip install transformers torch

  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl (207.5 MB)
  Attempting uninstall: nvidia-cusparse-cu12
    Found existing installation: nvidia-cusparse-cu12 12.5.1.3
    Uninstalling nvidia-cusparse-cu12-12.5.1.3:
      Successfully uninstalled nvidia-cusparse-cu12-12.5.1.3
  Attempting uninstall: nvidia-cublas-cu12
    Fo

In [2]:
# Disable WandB completely
import os
os.environ["WANDB_DISABLED"] = "true"

In [19]:
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
import pandas as pd

In [4]:
df = pd.read_csv('/content/twitter_training.csv', encoding='utf-8')

df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [5]:
df = df.iloc[:, 2:]

In [6]:
df.columns = ['label_text', 'text']

In [7]:
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df = df[df['label_text'].isin(label_map.keys())]
df['label'] = df['label_text'].map(label_map)

In [8]:
df = df.dropna(subset=['text'])
df['text'] = df['text'].astype(str)

In [9]:
df = df[df['label'].isin([0, 1, 2])]

In [10]:
from transformers import BertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [11]:
# Initialize the BERT tokenizer and model for sequence classification
tokenizer = BertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.

In [12]:
# Tokenization function
def preprocess(batch):
    return tokenizer(batch['text'], truncation=True, padding=True)

In [13]:
# Convert to HuggingFace dataset and tokenize
dataset = Dataset.from_pandas(df[['text', 'label']])
dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/61120 [00:00<?, ? examples/s]

In [14]:
# Train-test split
dataset = dataset.train_test_split(test_size=0.2)

In [20]:
def compute_metrics(pred):
    preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average='weighted')
    }

In [21]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    load_best_model_at_end=True,
    save_total_limit=1,
    save_strategy="epoch",
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-22-3051403215.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.154200,0.372222,0.901587,0.901730
2,0.122200,0.273655,0.931119,0.931240
3,0.058100,0.293685,0.937418,0.937503


TrainOutput(global_step=9168, training_loss=0.13099403208671023, metrics={'train_runtime': 3516.0473, 'train_samples_per_second': 41.72, 'train_steps_per_second': 2.607, 'total_flos': 1.0029252489686784e+16, 'train_loss': 0.13099403208671023, 'epoch': 3.0})

In [24]:
# # Evaluate the model
results = trainer.evaluate()
# print(f"Evaluation Results: {results}")

In [25]:
print(f"\nEvaluation Accuracy: {results['eval_accuracy']:.4f}")
print(f"Evaluation Loss: {results['eval_loss']:.4f}")


Evaluation Accuracy: 0.9311
Evaluation Loss: 0.2737
